In [1]:
import os 
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


/Users/jy/SKN23/multimodal/multi_venv/lib/python3.12/site-packages/tf_keras/src/datasets/cifar.py:32: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


In [ ]:
import tensorflow as tf                                              # 텐서플로우 라이브러리 임포트
from tensorflow import keras                                         # 케라스 API 임포트
from tensorflow.keras import layers                                  # 케라스 레이어 모듈 임포트
import tensorflow_model_optimization as tfmot                        # 모델 최적화 도구 임포트
from tensorflow_model_optimization.sparsity import keras as sparsity  # 프루닝 관련 모듈 임포트

# 1. 모델 생성
model = keras.Sequential([ 
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)), # 합성곱 레이어 (32 필터)
    layers.MaxPooling2D((2, 2)),                                           # 맥스 풀링 레이어
    layers.Flatten(),                                                      # 데이터 평탄화 레이어
    layers.Dense(64, activation='relu'),                                   # 완전 연결 레이어 (64 노드)
    layers.Dense(10, activation='softmax')                                 # 출력 레이어 (10 클래스)
])

# 2. 프루닝 스케줄 적용
pruned_model = sparsity.prune_low_magnitude(
    model,
    pruning_schedule=sparsity.PolynomialDecay(
        initial_sparsity=0.0,             # 초기 희소성 (0%)
        final_sparsity=0.5,               # 최종 희소성 (50%)
        begin_step=0,                     # 프루닝 시작 스텝
        end_step=1000                     # 프루닝 종료 스텝
    )
)

# 3. 프루닝 스텝 업데이트 콜백
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()   # 학습 중 프루닝 상태 업데이트
]

# 4. 모델 컴파일
pruned_model.compile(
    optimizer='adam',                          # Adam 옵티마이저
    loss='sparse_categorical_crossentropy',    # 손실 함수 설정
    metrics=['accuracy']                       # 평가지표: 정확도
)

# 5. 모델 학습
pruned_model.fit(
    x_train/255.0,                             # 입력 데이터 정규화 및 전달
    y_train,                                   # 학습용 정답 데이터
    epochs=2,                                  # 전체 학습 반복 횟수
    callbacks=callbacks                        # 프루닝 업데이트 콜백 적용
)

Epoch 1/2
1563/1563 [==============================] - 15s 9ms/step - loss: 1.5507 - accuracy: 0.4463
Epoch 2/2
1563/1563 [==============================] - 12s 8ms/step - loss: 1.2593 - accuracy: 0.5575


In [ ]:
# 양자화 (QAT)
quantize_model = tfmot.quantization.keras.quantize_model

quantize_model = quantize_model(model)

quantize_model.compile(
    optimizer = 'adam'
)